# Instant Gratification!


In this project, we will use the **OpenAI API** along with the **BeautifulSoup** library to scrape web content and generate summaries based on the websites we provide.

This is our first mini-project, designed to build familiarity with cutting-edge large language model (LLM) APIs and prompt engineering techniques.

In [8]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import openai


In [35]:
# use openai api
api_key = "enter your own api key"
openai.api_key = api_key


In [13]:
# define website class, with three attributes 
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        # Fetching and Parsing the Webpage Content 
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        # exclude irrelevant content
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        # extract text content
        self.text = soup.body.get_text(separator="\n", strip=True)

In [17]:
# Let's try one out

ed = Website("https://www.rbc.com/our-company/")
print(ed.title)
print(ed.text)

Our Company - RBC
Skip to main content
Other Sites
Search RBC...
Contact Us
Find a Branch
Call
1-800-769-2511
EN
Sign In
SIGN IN
MENU
Ask your question
Our Company
Our Impact
Investor Relations
News & Stories
Careers
Thought Leadership
Search RBC
Personal Banking
Other Sites
Promotions
Contact Us
Language
AdChoices
MENU
Our Company
Helping Clients Thrive and Communities Prosper
RBC is a global financial institution with a purpose-driven, principles-led approach to delivering leading performance, and creating value for our clients and communities.
What We Value
We’re on a purpose-driven journey to build the RBC of the future and reimagine the role we play in the lives of our customers. Our purpose inspires us every day to bring our best and use our imagination and insights to build a better future for our clients and communities.
Learn More
Our Businesses
For more than 150 years, we’ve gone where our clients have gone – expanding across Canada, the United States, and to select global ma

# Types of prompts


A system prompt that tells them what task they are performing and what tone they should use

A user prompt -- the conversation starter that they should reply to

In [22]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [24]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [29]:
# pass in the prompts we just wrote
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

Time to bring it together - the API for OpenAI is very simple!


In [31]:
def summarize(url):
    # use Website class, return website body text
    website = Website(url)
    # openai api
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [37]:
summarize("https://www.rbc.com/our-company/")


"# Summary of Our Company - RBC\n\nThe RBC (Royal Bank of Canada) website presents the bank as a global financial institution that prioritizes purpose-driven and principles-led approaches to business. With over 150 years of experience, RBC serves 17 million clients across Canada, the U.S., and other international markets through various segments including personal banking, wealth management, and capital markets.\n\n## Key Highlights\n\n- **Mission**: RBC is committed to helping clients thrive and communities prosper, focusing on building a sustainable future.\n- **Values**: The organization emphasizes a purpose-driven journey to reimagine its role in the lives of customers.\n- **Corporate Governance**: RBC maintains a high standard of corporate governance, fostering a proactive culture throughout the organization.\n- **Workforce**: The bank employs over 97,000 individuals across different countries.\n- **Awards**: RBC has received several accolades, including being named the 'North Ame

In [39]:
# uses IPython.display.Markdown to format and display the summary in Markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [41]:
display_summary("https://www.rbc.com/our-company/")


# Summary of Our Company - RBC

The "Our Company" section of the RBC website outlines the institution's commitment to helping clients thrive and communities prosper through a purpose-driven approach. RBC is a significant global financial institution, with a history spanning over 150 years and a strong presence in Canada, the United States, and various international markets. 

## Key Highlights

- **Purpose and Values:** RBC's mission focuses on building a better future for clients and communities through innovation and insights.
- **Business Segments:** The company operates five key business segments and serves approximately 17 million clients across different countries.
- **Leadership:** Information about RBC's senior leadership is available, highlighting the strategic direction of the organization.
- **Corporate Governance:** RBC emphasizes a strong governance culture with industry-leading standards.
- **Quick Facts:** 
  - Operates under the master brand name RBC.
  - One of the largest banks globally by market capitalization.
  - Employs over 97,000 staff.
- **Awards and Recognition:** RBC has received numerous accolades, including being named 'North American Retail Bank of the Year' for the third consecutive year, among others.

## Latest News
RBC's newsroom includes insights and thought leadership; however, specific recent news updates are not detailed in this section.

Overall, the content reflects RBC's dedication to performance, trust, and value creation, along with their commitment to clients and the broader community.

# Let's try more websites


In [45]:
display_summary("https://www.bnnbloomberg.ca/")


# BNN Bloomberg – Canada Business News Overview

BNN Bloomberg provides comprehensive coverage of Canadian business news, financial markets, and economic updates. The site features a variety of sections dedicated to investing, market performance, company news, economics, real estate, and personal finance.

## Key Highlights:

- **Interest Rates & Economic Insights**: Veteran economist David Rosenberg advocates for the Bank of Canada to implement continuous cuts of 50 basis points to address the balance of demand and supply in the economy. 

- **Government Finance**: The federal government has reported a substantial deficit of $9.8 billion for the period from April to August 2024.

- **Corporate Updates**:
  - **Corus Entertainment**: The company has signed an amended debt agreement with banks and reported a loss of $26 million in Q4.
  - **Brookfield Unit**: Plans to divest $1.5 billion in private credit assets are underway.
  
- **Market Reactions**: Commentary on the impact of the Bank of Canada's recent rate cuts, with a focus on implications for small businesses and the real estate sector.

- **Videos & Analysis**: The platform includes a range of videos discussing topics such as small business needs, economic impact of labor dynamics, and investment strategies amidst market fluctuations.

For more in-depth insights, consider the top picks and expert analyses featured on the platform regularly.